In [1]:
import numpy as np
import cv2
import glob
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.metrics import silhouette_score
from numpy import savetxt
import shutil
from tensorflow import keras
import tensorflow as tf


tf.config.get_visible_devices()


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [2]:
image_train_gen = keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.2,
                                                         height_shift_range=0.2,
                                                         rescale=1 / 255.,
                                                         horizontal_flip=True,
                                                         fill_mode="nearest",
                                                         validation_split=0.2
                                                         )

image_test_gen = keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.2,
                                                         height_shift_range=0.2,
                                                         rescale=1 / 255.,
                                                         horizontal_flip=True,
                                                         fill_mode="nearest"
                                                         )

In [ ]:
train_images = image_train_gen.flow_from_directory(
    directory="laser/train/",
    color_mode="rgb",
    class_mode="binary",
    subset='training',
    seed=42
    )

Found 57555 images belonging to 2 classes.


In [ ]:
valid_images = image_train_gen.flow_from_directory(
    directory="laser/train/",
    color_mode="rgb",
    class_mode="binary",
    subset='validation',
    seed=42
    )

Found 14388 images belonging to 2 classes.


In [ ]:
test_images = image_test_gen.flow_from_directory(
    directory="laser/test/",
    color_mode="rgb",
    class_mode="binary",
    seed=42
    )

Found 17984 images belonging to 2 classes.


In [ ]:
train_images.image_shape

(256, 256, 3)

In [2]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[256,256,3]))
model.add(keras.layers.Dense(1000, activation="relu"))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))  # "sigmoid" for binary classification

In [ ]:
model.compile(loss="binary_crossentropy",   ## for binary we use "binary_crossentropy"
              optimizer="sgd",    ###  optimizer=keras.optimizers.SGD(lr=???) for learnin grate 
              metrics=["accuracy"])

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 196608)            0         
_________________________________________________________________
dense (Dense)                (None, 1000)              196609000 
_________________________________________________________________
dropout (Dropout)            (None, 1000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               300300    
_________________________________________________________________
dense_2 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 1

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
checkpoint_cb = keras.callbacks.ModelCheckpoint("models/model-{epoch:02d}-{val_accuracy:.2f}.hdf5",  monitor='val_accuracy', save_best_only=True)

In [ ]:
history = model.fit(train_images, epochs=100, validation_data=valid_images, callbacks=[early_stopping_cb, checkpoint_cb])